# Title

In [1]:
from utils.test_file import generate_group_sequence
from algorithms.online import FirstFit, BestFit, WorstFit, MinCovidChairs, Hybrid_BF_CC
import pandas as pd
import pickle
import plotly.express as px
import itertools
from online_batch import run_algorithm_with_original_groups, repeat_algorithm_with_different_groups, get_file_names

## Set variables

We need to define:

* which files (=grids) we are going to run
* how many random groups we gonna test per grids
* the number of groups
* which algorithms

In [2]:
FILE_DIR = "input/online"
# 100 different group sequences from each 50 groups long
GROUPS_LIST = [generate_group_sequence(50) for i in range(10)]
ALGORITHMS = [FirstFit, BestFit, WorstFit, MinCovidChairs, Hybrid_BF_CC]

Gather all the files that we want to run on.

In [3]:
file_names = get_file_names(FILE_DIR)

Run the algorithms and gather the results in a dataframe

In [4]:
%%capture
result = {}
for file in file_names:
    grid_path = f"{FILE_DIR}/{file}"

    grid_result = {}
    for algorithm in ALGORITHMS:
        alg_filled_chairs = []
        alg_filled_chairs.append(run_algorithm_with_original_groups(algorithm, grid_path))
        alg_filled_chairs = alg_filled_chairs + repeat_algorithm_with_different_groups(algorithm, grid_path, GROUPS_LIST)
        alg_name = str(algorithm.__name__)
        grid_result.update({alg_name: alg_filled_chairs})
    
    result.update({file: grid_result})


In [5]:
print("The following grids were solved:")    
print(result.keys())

The following grids were solved:
dict_keys([&#39;Online1.txt&#39;, &#39;Online10.txt&#39;, &#39;Online11.txt&#39;, &#39;Online12.txt&#39;, &#39;Online2.txt&#39;, &#39;Online3.txt&#39;, &#39;Online4.txt&#39;, &#39;Online5.txt&#39;, &#39;Online6.txt&#39;, &#39;Online7.txt&#39;, &#39;Online8.txt&#39;, &#39;Online9.txt&#39;])


Put everything in a dataframe:

In [6]:
df_list = []
for grid, algs in result.items():
    alg_series = []
    for alg, value_list in algs.items():
        alg_series.append(pd.Series(value_list, name=alg))
    df = pd.DataFrame(alg_series).transpose()
    df = df.assign(grid=grid[0:-4]).set_index('grid', append=True, drop=True)
    df_list.append(df)
df = pd.concat(df_list)

Let's extract the records with the original groups from the input grid files, and put it in a seperate dataframe

In [7]:
# The first group is the original group included in the grid file
df_real_groups = df.xs(0, level=None)
df_real_groups

,FirstFit,BestFit,WorstFit,MinCovidChairs,Hybrid_BF_CC
grid,,,,,
Online1,5,5,5,5,5
Online10,74,67,73,79,67
Online11,45,52,39,52,52
Online12,144,147,120,141,147
Online2,9,9,5,5,9
Online3,15,11,13,15,15
Online4,12,12,12,16,16
Online5,22,26,22,22,26
Online6,29,29,27,33,33


And export the dataframe

In [8]:
df_real_groups.to_pickle("results/online_results.p")

We also make a separate dataframe for the generated group simulation records: 

In [9]:
# All the groups except group [0] belongs to the simulation
df_sim = df.loc[pd.IndexSlice[range(1,len(GROUPS_LIST)+1),:]]
df_sim

FirstFit  BestFit  WorstFit  MinCovidChairs  Hybrid_BF_CC
   grid                                                               
1  Online1          5        5         5               5             5
   Online10        67       67        64              72            67
   Online11        57       59        51              61            61
   Online12       130      140       115             141           132
   Online2          8        8         8               8             8
...               ...      ...       ...             ...           ...
10 Online5         25       24        23              25            27
   Online6         28       30        25              30            32
   Online7         41       41        33              39            40
   Online8         44       45        43              46            45
   Online9         66       64        60              68            63

[120 rows x 5 columns]

The following analyses are on the simulation results.

In [40]:
df_sim_mean = df_sim.groupby(['grid']).mean().reset_index().melt(id_vars='grid', value_name='seats', var_name='algorithm')
# df_sim_avg

,grid,algorithm,seats
0,Online1,FirstFit,4.9
1,Online10,FirstFit,69.8
2,Online11,FirstFit,56.3
3,Online12,FirstFit,130.6
4,Online2,FirstFit,7.3
5,Online3,FirstFit,15.4
6,Online4,FirstFit,25.3
7,Online5,FirstFit,23.3
8,Online6,FirstFit,29.3
9,Online7,FirstFit,37.9


In [33]:
df_sim_std = df_sim.groupby(['grid']).std().reset_index().melt(id_vars='grid', value_name='std', var_name='algorithm')
# df_sim_std

,grid,algorithm,std
0,Online1,FirstFit,1.100505
1,Online10,FirstFit,2.820559
2,Online11,FirstFit,2.668749
3,Online12,FirstFit,4.812022
4,Online2,FirstFit,0.948683
5,Online3,FirstFit,0.966092
6,Online4,FirstFit,1.494434
7,Online5,FirstFit,1.337494
8,Online6,FirstFit,2.359378
9,Online7,FirstFit,2.330951


In [41]:
df_sim_comb = df_sim_mean.merge(df_sim_std, on=['grid', 'algorithm'])
# df_sim_comb

,grid,algorithm,seats,std
0,Online1,FirstFit,4.9,1.100505
1,Online10,FirstFit,69.8,2.820559
2,Online11,FirstFit,56.3,2.668749
3,Online12,FirstFit,130.6,4.812022
4,Online2,FirstFit,7.3,0.948683
5,Online3,FirstFit,15.4,0.966092
6,Online4,FirstFit,25.3,1.494434
7,Online5,FirstFit,23.3,1.337494
8,Online6,FirstFit,29.3,2.359378
9,Online7,FirstFit,37.9,2.330951


The mean performance of each algorithm per grid:

In [42]:
bar_mean = px.bar(df_sim_comb, x='grid', color='algorithm', y='seats', barmode='group', error_y='std')
bar_mean.show()

Best performance of each algorithm per grid:

In [43]:
df_sim_max = df_sim.groupby(['grid']).max().reset_index().melt(id_vars='grid', value_name='seats', var_name='algorithm')
px.bar(df_sim_max, x='grid', color='algorithm', y='seats', barmode='group').show()

### Ranking of algorithm in respect to each other per grid?

In [51]:
# For each grid - group_sequence combination, rank the algorithms in respect to each other
df_sim.index.names = ['group_sequence', 'grid']
df_ranking = df_sim.reset_index().melt(id_vars=['group_sequence','grid'], value_name='seats', var_name='algorithm').sort_values(['grid', 'group_sequence'])
rank = df_ranking.groupby(['grid', 'group_sequence'])
rank = rank['seats'].rank(method='min', ascending=False)
df_ranking['rank'] = rank
df_ranking

,group_sequence,grid,algorithm,seats,rank
0,1,Online1,FirstFit,5,1.0
120,1,Online1,BestFit,5,1.0
240,1,Online1,WorstFit,5,1.0
360,1,Online1,MinCovidChairs,5,1.0
480,1,Online1,Hybrid_BF_CC,5,1.0
...,...,...,...,...,...
119,10,Online9,FirstFit,66,2.0
239,10,Online9,BestFit,64,3.0
359,10,Online9,WorstFit,60,5.0
479,10,Online9,MinCovidChairs,68,1.0


In [52]:
# initialize scoreboard dataframe
df_ranking_result = pd.DataFrame(
    list(itertools.product(
        df_ranking['grid'].unique(), df_ranking['algorithm'].unique()
        ))
        ,columns=['grid', 'algorithm']).assign(**{'0':0,'1':0,'2':0,'3':0,'4':0,'5':0})
df_ranking_result

,grid,algorithm,0,1,2,3,4,5
0,Online1,FirstFit,0,0,0,0,0,0
1,Online1,BestFit,0,0,0,0,0,0
2,Online1,WorstFit,0,0,0,0,0,0
3,Online1,MinCovidChairs,0,0,0,0,0,0
4,Online1,Hybrid_BF_CC,0,0,0,0,0,0
5,Online10,FirstFit,0,0,0,0,0,0
6,Online10,BestFit,0,0,0,0,0,0
7,Online10,WorstFit,0,0,0,0,0,0
8,Online10,MinCovidChairs,0,0,0,0,0,0
9,Online10,Hybrid_BF_CC,0,0,0,0,0,0
